# MScFE 690 - Capstone Project Codes
Group 6
8 February 2022

In [1]:
#Import Packages

import numpy as np
import pandas as pd
import statsmodels.regression.linear_model as lm
import statsmodels.tools.tools as ct
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import statsmodels.api as sm

import yfinance as yf
from pandas_datareader import data
import pandas_datareader as pdr
import datetime
import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import common databases

Damodaran=pd.ExcelFile('ctryprem.xlsx')
RWA_tab= pd.read_excel(Damodaran, 'Regional Weighted Averages',header=0,index_col=0)
RWA_tab = RWA_tab.dropna(subset=['Equity Risk Premium'], axis=0)

#RWA_tab.head()

# List of GICS sectors
sector_map={'Utilities':'XLU', 'Technology':'XLK', 'Basic Materials':'XLB', 'Industrials':'XLI', 'Healthcare': 'XLV',
           'Financials': 'XLF', 'Financial Services': 'XLF', 'Energy':'XLE', 'Consumer Staples':'XLP', 'Consumer Defensive':'XLP',
           'Consumer Discretionary':'XLY', 'Consumer Cyclical':'XLY',
           'Communication Services':'XLC', 'Real Estate':'XLRE'}

# Risk free interest rate proxy list
Rf_map={'30y':'^TYX','10y':'^TNX','3m':'^IRX'}

#List of countries and econ status
country_status=pd.read_csv('DevelopedCountries.csv',header=0)

## 0. User inputs

In [3]:
# User inputs
print('+++++++++ You are about to enter inputs for the models ...... ')
# Ticker of interest
ticker= input('==1. Please input the stock of interest for beta calculation then press Enter: ')

#Reference ticker in US market (if the stock is in an emerging market)
ref_ticker=input('==2. For Modified CAPM, please input the reference stock in US market for beta calculation then press Enter: ')

#Country of the stock
country=input('==3. Please input country of the interested stock then press Enter: ')

#Economic status of the country
if country_status[country_status.country==country]['hdi2019'].values >= 0.80:
    econ_status = 'developed market'
else:
    econ_status = 'emerging market'

#Sector - automatically identified based on Ticker
if econ_status == 'developed market':
    sector = yf.Ticker(ticker).info['sector']
else:
    sector=yf.Ticker(ref_ticker).info['sector']
sector = [sector,sector_map[sector]]

#Risk Free interest rate proxy for APT
rf_select=input('==4. Please input the type of risk free interest rate proxy, ie. (30y / 10y / 3m) T-bond: ')

if (rf_select!='30y')&(rf_select!='10y')&(rf_select!='3m'):
    print('ArgumentError: rf can only accept 30y, 10y, or 3m.')

# Beta of sector in each market for International CAPM
beta_iM=[]
countries=[]

i=1
while True:
    country_name=input('==5.{} For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: '.format(i))
    countries.append(country_name)
    i+=1
    if countries[-1] !='Q':
        beta_iM.append(1+RWA_tab.loc[countries[-1],'Equity Risk Premium'])
    else:
        break
        
#User to provide alphas for International CAPM
alphas=[]

selfput=input('==6. For Modified International CAPM, do you want to input sector weight by country (Y/N): ')
if selfput =='Y':
    for i in range(len(beta_iM)):
        alp=input('==6.{} For Modified International CAPM, please input sector weight per country in decimal: '.format(i+1))
        alphas.append(float(alp))
        


#Country risk premium - automatically retrieved based on country.
if country not in RWA_tab.index:
    CR=RWA_tab.loc['Global','Country Risk Premium']
else:
    CR=RWA_tab.loc[country,'Country Risk Premium']
    
# User to select the number of factors for Fama French model
factor=int(input('==7. For Fama French model, please input the number of factors to be considered (3 or 5): '))
if (factor!=3) & (factor!=5):
    print('ArgumentError: Factor must be either 3 or 5.')

print('+++++++++ Thank you, the inputs have been registered!')

+++++++++ You are about to enter inputs for the models ...... 
==1. Please input the stock of interest for beta calculation then press Enter: BHP
==2. For Modified CAPM, please input the reference stock in US market for beta calculation then press Enter: NEM
==3. Please input country of the interested stock then press Enter: Australia
==4. Please input the type of risk free interest rate proxy, ie. (30y / 10y / 3m) T-bond: 10y
==5.1 For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: Australia
==5.2 For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: United States
==5.3 For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: South Africa
==5.4 For Modified International CAPM, please input country(ies) dominating the sector of interest then press En

In [3]:
# User inputs
print('+++++++++ You are about to enter inputs for the models ...... ')
# Ticker of interest
ticker= input('==1. Please input the stock of interest for beta calculation then press Enter: ')

#Reference ticker in US market (if the stock is in an emerging market)
ref_ticker=input('==2. For Modified CAPM, please input the reference stock in US market for beta calculation then press Enter: ')

#Country of the stock
country=input('==3. Please input country of the interested stock then press Enter: ')

#Economic status of the country
if country_status[country_status.country==country]['hdi2019'].values >= 0.80:
    econ_status = 'developed market'
else:
    econ_status = 'emerging market'

#Sector - automatically identified based on Ticker
if econ_status == 'developed market':
    sector = yf.Ticker(ticker).info['sector']
else:
    sector=yf.Ticker(ref_ticker).info['sector']
sector = [sector,sector_map[sector]]

#Risk Free interest rate proxy for APT
rf_select=input('==4. Please input the type of risk free interest rate proxy, ie. (30y / 10y / 3m) T-bond: ')

if (rf_select!='30y')&(rf_select!='10y')&(rf_select!='3m'):
    print('ArgumentError: rf can only accept 30y, 10y, or 3m.')

# Beta of sector in each market for International CAPM
beta_iM=[]
countries=[]

i=1
while True:
    country_name=input('==5.{} For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: '.format(i))
    countries.append(country_name)
    i+=1
    if countries[-1] !='Q':
        beta_iM.append(1+RWA_tab.loc[countries[-1],'Equity Risk Premium'])
    else:
        break
        
#User to provide alphas for International CAPM
alphas=[]

selfput=input('==6. For Modified International CAPM, do you want to input sector weight by country (Y/N): ')
if selfput =='Y':
    for i in range(len(beta_iM)):
        alp=input('==6.{} For Modified International CAPM, please input sector weight per country in decimal: '.format(i+1))
        alphas.append(float(alp))
        


#Country risk premium - automatically retrieved based on country.
if country not in RWA_tab.index:
    CR=RWA_tab.loc['Global','Country Risk Premium']
else:
    CR=RWA_tab.loc[country,'Country Risk Premium']
    
# User to select the number of factors for Fama French model
factor=int(input('==7. For Fama French model, please input the number of factors to be considered (3 or 5): '))
if (factor!=3) & (factor!=5):
    print('ArgumentError: Factor must be either 3 or 5.')

print('+++++++++ Thank you, the inputs have been registered!')

+++++++++ You are about to enter inputs for the models ...... 
==1. Please input the stock of interest for beta calculation then press Enter: BHP
==2. For Modified CAPM, please input the reference stock in US market for beta calculation then press Enter: NEM
==3. Please input country of the interested stock then press Enter: Australia
==4. Please input the type of risk free interest rate proxy, ie. (30y / 10y / 3m) T-bond: 10y
==5.1 For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: Australia
==5.2 For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: United States
==5.3 For Modified International CAPM, please input country(ies) dominating the sector of interest then press Enter to proceed or Q to end: South Africa
==5.4 For Modified International CAPM, please input country(ies) dominating the sector of interest then press En

# 1. Results

Please load Section 2 below first before running Results

In [21]:
res=[[*ModifiedCAPM()[:3]],[*IntlCAPM()[:3]],[*apt()[:3]],[*famafrench()[:3]]]
pd.DataFrame(res,columns =['Annualized Expected Return (%)', 'R-squared (%)', 'MSE'],index=['Modified CAPM','International CAPM','APT','Fama French {}'.format(factor)]).round(2)

,Annualized Expected Return (%),R-squared (%),MSE
Modified CAPM,-8.76,99.91,0.00
International CAPM,-837.65,99.99,0.00
APT,2160.36,18.16,0.52
Fama French 3,70.44,26.25,0.12


## 2.  Models 

### 2.1. Modified CAPM

$E(Ri)=rf+βi×[E(RM)−rf]+CR$

In [4]:
def ModifiedCAPM():
    
    global ticker
    global rf_select
    global CR
    global econ_status
    
    #Find risk free interest rate
    
    rf = yf.download(tickers=Rf_map[rf_select], period='5y', interval='1d',progress=False)[['Adj Close']]
    rf = rf[['Adj Close']].resample('M').mean()
    rf.columns = ['rf']
    
    #calculate return and excess return
    if econ_status == 'developed market':
        close=yf.download(ticker, period='5y', interval='1d',progress=False)['Adj Close']
    else:
        close=yf.download(ref_ticker, period='5y', interval='1d',progress=False)['Adj Close']
        
    close = close.resample('M').mean().pct_change().dropna()
    data = pd.DataFrame(close).merge(rf, how='left', left_index=True, right_index=True)
    data['excessReturn'] = data['Adj Close'] - data['rf']
    
    #calculate excess market return
    market=yf.download('SPY', period='5y', interval='1d',progress=False)['Adj Close']
    market=market.resample('M').mean().pct_change().dropna()
    data['excessMarket']=market-data['rf']
    
    
    #Regressing beta
    
    ols = sm.OLS(data.excessReturn,data.excessMarket).fit()
    #pred = ols.predict(data.excessMarket)
    #mse=mean_squared_error(data.excessReturn, pred)
    
    return (data['rf'][-1] + ols.params[0]*data.excessMarket[-1]+CR)*100*12, ols.rsquared*100,ols.mse_resid,ols
    

In [6]:
#Testing
print(ModifiedCAPM()[2],ModifiedCAPM()[3].mse_resid)
print(ModifiedCAPM()[-1].summary())

0.0036359083178919126 0.0036359088443738757
                                 OLS Regression Results                                
Dep. Variable:           excessReturn   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          6.920e+04
Date:                Sun, 06 Feb 2022   Prob (F-statistic):                    2.88e-92
Time:                        22:08:20   Log-Likelihood:                          83.875
No. Observations:                  60   AIC:                                     -165.7
Df Residuals:                      59   BIC:                                     -163.7
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.

### 2.2. Modified International CAPM

$E[R_i]=R_f+β_P×(E[R_M ]-R_f )$

In [15]:
def IntlCAPM():
    
    global ticker
    global rf_select
    global CR
    global econ_status
    global sector
    global beta_iM
    global country
    global alphas
    
    #Find risk free interest rate
    
    rf = yf.download(tickers=Rf_map[rf_select], period='5y', interval='1d',progress=False)[['Adj Close']]
    rf = rf[['Adj Close']].resample('M').mean()
    rf.columns = ['rf']
    
    #calculate return and excess return of the sector

    close=yf.download(sector[1], period='5y', interval='1d',progress=False)['Adj Close']
        
    close = close.resample('M').mean().pct_change().dropna()
    data = pd.DataFrame(close).merge(rf, how='left', left_index=True, right_index=True)
    data['excessSectorReturn'] = data['Adj Close'] - data['rf']
    
    #calculate excess market return
    market=yf.download('SPY', period='5y', interval='1d',progress=False)['Adj Close']
    market=market.resample('M').mean().pct_change().dropna()
    data['excessMarket']=market-data['rf']
    
    #Regressing beta_tM
    
    ols = sm.OLS(data.excessSectorReturn,data.excessMarket).fit()
    #pred = ols.predict(data.excessMarket)
    #mse=mean_squared_error(data.excessSectorReturn, pred)
    
    beta_tM=ols.params[0]
    
    #Calculate beta_tnM and beta_tmM
    beta_tiM=[beta_tM*element for element in beta_iM]
    
    #Generate alphas
    if len(alphas)==0:
        alphas=np.random.dirichlet(np.ones(len(beta_tiM)))
    
    #Calculate beta_p
    beta_p=alphas@np.array(beta_tiM)
    beta_p
    
    #Calculate modified international CAMP
    Int_R=data.rf[-1]+beta_p*data.excessMarket[-1]
    

    return  Int_R*100*12,ols.rsquared*100, ols.mse_resid, ols
    


In [19]:
# Testing
print(IntlCAPM()[:3])
print(IntlCAPM()[-1].summary())

(-837.6493605628109, 99.98894221338941, 0.00047469658503189845)
                                 OLS Regression Results                                
Dep. Variable:     excessSectorReturn   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          5.335e+05
Date:                Sun, 06 Feb 2022   Prob (F-statistic):                   2.01e-118
Time:                        22:10:51   Log-Likelihood:                          144.95
No. Observations:                  60   AIC:                                     -287.9
Df Residuals:                      59   BIC:                                     -285.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|

### 2.3.  APT

$E(R_i )=R_f+β_{i1}×f_1+β_{i2}×f_2+β_{i3}×f_3+⋯+β_{i5}×f_5$

In [11]:
def apt():
    
    global rf_select
    global econ_status
    global country
    global sector
    global ticker
    global ref_ticker


    # load risk free interest rate
    rf = yf.download(tickers=Rf_map[rf_select], period='5y', interval='1d',progress=False)[['Adj Close']]
    rf = rf[['Adj Close']].resample('M').mean()
    rf.columns = ['rf']
    
    #load factors data
    indicator_list = ['CPIEALL', 'INDPRO', 'FEDFUNDS']
    indicator = pdr.DataReader(indicator_list, 'fred')
    indicator = indicator.resample('M').mean().pct_change()
    indicator.columns = ['Inflation', 'IndustrialProduction', 'FedFunds']
    
    sector_data=yf.download(sector[1], period='5y', interval='1d',progress=False)['Adj Close']
    sector_data = sector_data.resample('M').mean().pct_change().dropna()
    indicator[sector[1]] = sector_data
    
    #prepare stock data
    #print('Downloading Data')
    if econ_status == 'developed market':
        close=yf.download(ticker, period='5y', interval='1d',progress=False)['Adj Close']
    else:
        close=yf.download(ref_ticker, period='5y', interval='1d',progress=False)['Adj Close']
    #print()
    
    #calculate return and excess return
    close = close.resample('M').mean().pct_change().dropna()
    data = pd.DataFrame(close).merge(rf, how='left', left_index=True, right_index=True)
    #print(data.head())
    data['excessReturn'] = data['Adj Close'] - data['rf']
    data = data.merge(indicator, how='left', left_index=True, right_index=True)
    
    #country risk
    data['CR'] = RWA_tab[RWA_tab.index==country]['Country Risk Premium'][0]
    data = data[['Inflation', 'IndustrialProduction', 'FedFunds', sector[1], 'CR', 'excessReturn']].dropna()
    
    
    #print('\nData:')
    #print(data.head())
    #print()
    
    #fit data
    formula =  "excessReturn~ Inflation+IndustrialProduction+FedFunds+{}+CR".format(sector[1])
    model = lm.OLS.from_formula(formula, data = data).fit()
    
    #evaluate model
    r2 = model.rsquared
    mse = model.mse_resid
    E_return=model.params[1:]@data.iloc[-1,:-1]+rf.iloc[-1]
    return E_return[0]*12*100,r2*100,mse,model

In [12]:
print(apt()[:3])
# R2<0 or >1 means the data fits really poorly https://stats.stackexchange.com/questions/334004/can-r2-be-greater-than-1'
print(apt()[-1].summary())

(2160.361827604576, 18.155413077986527, 0.5194788611998075)
                            OLS Regression Results                            
Dep. Variable:           excessReturn   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     2.939
Date:                Sun, 06 Feb 2022   Prob (F-statistic):             0.0287
Time:                        22:09:29   Log-Likelihood:                -60.691
No. Observations:                  58   AIC:                             131.4
Df Residuals:                      53   BIC:                             141.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

C:\Users\nikolaus\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


### 2.4. Fama French

- Three factors: $E(R_i )-R_f=β_1×(E[R_M ]-R_f )+β_2×SMB+β_3×HML$
- Five factors: $E(R_i )-R_f=β_1×(E[R_M ]-R_f )+β_2×SMB + β_3×HML + \beta_4 \times RWM + \beta_5 \times CMA $


In [17]:
def famafrench():
    '''Return a 3 or 5 factor Fama French model for the chosen stock along with the evaluation metrics.
    
    Arguments
    stock: ticker of the stock
    factor: 3 (default), 5
    '''
    global factor
    global econ_status
    global country
    global ticker
    global ref_ticker
    
    enddate = dt.datetime.strptime("2021-12-31", "%Y-%m-%d").date()
    startdate = enddate - dt.timedelta(days=365*5)
    #check factors, return error message if it is not 3 or 5
    if factor==5:
        #print('5 Factor Fama-French\n\n')
        FF = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.csv', header=2, index_col=0).dropna()
        FF.index = pd.to_datetime(FF.index, format='%Y%m%d')
        FF.columns = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
    if factor==3:  
        #print('3 Factor Fama-French\n\n')
        FF=pd.read_csv('F-F_Research_Data_Factors_Daily.csv', header=3, index_col=0).dropna()
        FF.index = pd.to_datetime(FF.index, format='%Y%m%d')
        FF.columns = ['Mkt_RF', 'SMB', 'HML', 'RF']

    
    #prepare stock data
    #print('Downloading Data')
    if econ_status == 'developed market':
        close=yf.download(ticker, start=startdate,end=enddate, interval='1d', progress=False)['Adj Close']
    else:
        close=yf.download(ref_ticker, start=startdate,end=enddate, interval='1d',progress=False)['Adj Close']
    #print()
    
    
    #calculate return and excess return
    close = pd.DataFrame(close).pct_change().dropna()
    data = close[['Adj Close']].merge(FF, how='left', left_index=True, right_index=True)
    
    #country risk
    if country not in RWA_tab.index:
        country = 'Global'
    data['CR'] = RWA_tab[RWA_tab.index==country]['Country Risk Premium'][0]
    data = data.resample('M').mean().dropna()
    data['excessReturn'] = data['Adj Close']*100 - data['RF']
    RF=data['RF'].copy()
    data = data[['Mkt_RF', 'SMB', 'HML', 'CR', 'excessReturn']].dropna()
    
    
    #fit model
    formula =  "excessReturn~ {}".format('+'.join(data.columns[:-1]))
    model = lm.OLS.from_formula(formula, data = data).fit()
    
    #evaluate model
    r2 =  model.rsquared
    mse =  model.mse_resid
    E_return=model.params[1:]@data.iloc[-1,:-1]+RF.iloc[-1]
    
    return E_return*252,r2*100,mse,model

In [18]:
# Testing
print(famafrench()[:3])
print(famafrench()[-1].summary())

(70.44331937893283, 26.250850784654446, 0.12355338941987562)
                            OLS Regression Results                            
Dep. Variable:           excessReturn   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     6.407
Date:                Sun, 06 Feb 2022   Prob (F-statistic):           0.000858
Time:                        22:10:17   Log-Likelihood:                -19.585
No. Observations:                  58   AIC:                             47.17
Df Residuals:                      54   BIC:                             55.41
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

C:\Users\nikolaus\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
